In [155]:
import pickle as pkl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from data_utils import results2df
from torchmetrics.functional.classification import multilabel_f1_score
import os

plt.style.use("science")

In [156]:
train_results = "../dataset/results/model=slow_r50_ds=panaf_seq_fd_only_feats=train_feats.pkl"
val_results = "../dataset/results/model=slow_r50_ds=panaf_seq_fd_only_e=200_feats=val_feats.pkl"
#train_results = "../dataset/results_old/training_progression/train/model=slow_r50_feats=epoch-100_split=train.pkl"
#val_results = "../dataset/results_old/training_progression/validation/model=slow_r50_feats=epoch-100_split=val.pkl"
metadata_file = "../dataset/metadata/new_metadata.csv"
behaviours_file = "../dataset/metadata/behaviours.txt"
segments_file = "../dataset/metadata/segments.txt"

camera_loc_df = pd.read_csv("../dataset/metadata/ordered_locations.txt", header=None)

# convert to list
camera_loc_list = camera_loc_df.values.tolist()
camera_loc_list = [loc[0] for loc in camera_loc_list]

with open(train_results, "rb") as f:
    train_data = pkl.load(f)

with open(
    val_results,
    "rb",
) as f:
    val_data = pkl.load(f)

metadata_df = pd.read_csv(metadata_file)

with open(behaviours_file, "rb") as f:
    behaviours = [beh.decode("utf-8").strip() for beh in f.readlines()]

with open(segments_file, "rb") as f:
    segments = [seg.decode("utf-8").strip() for seg in f.readlines()]

In [157]:
def count_videos_per_camera_behaviour(
    df: pd.DataFrame, camera_loc_list: list, num_labels: int = 14
):
    camera_loc_df = pd.DataFrame(camera_loc_list, columns=["utm"])
    df_count = pd.concat(
        [df.drop(columns="label"), df["label"].apply(pd.Series)],
        axis=1,
    )
    df_count = (
        df_count.groupby("utm")[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]
        .sum()
        .reset_index()
    )
    df_count.columns = ["utm"] + [str(i) for i in range(num_labels)]
    df_count = df_count.merge(camera_loc_df, on="utm", how="right").fillna(0)
    df_count.iloc[:, 1:] = df_count.iloc[:, 1:].astype(int)
    return df_count

In [158]:
train_df, val_df = results2df(train_data, val_data, metadata_df)

print(train_df.shape, val_df.shape)
diff = set(val_df.utm.unique()) - set(train_df.utm.unique())
exclude_utm = list(diff)
# remove utms from new_val_df that are not in new_train_df
val_df = val_df[~val_df["utm"].isin(exclude_utm)]

print(train_df.columns)

train_camera_loc = train_df["utm"].unique().tolist()
val_camera_loc = val_df["utm"].unique().tolist()


# count number of videos per camera location and add to dataframe
train_video_count = (
    train_df.groupby("utm")["name"]
    .count()
    .reset_index()
    .rename(columns={"name": "video_count"})
)
val_video_count = (
    val_df.groupby("utm")["name"]
    .count()
    .reset_index()
    .rename(columns={"name": "video_count"})
)

train_beh_count = count_videos_per_camera_behaviour(train_df, train_camera_loc)
val_beh_count = count_videos_per_camera_behaviour(val_df, val_camera_loc)


/home/kukushkin/imslowfast/notebooks/data_utils.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  df["pred"] = df.pred.apply(lambda x: torch.sigmoid(torch.tensor(x)))


(2848, 24) (1003, 24)
Index(['name', 'split', 'pred', 'feat', 'negative_x', 'subject_id',
       'subject_id_bg', 'label', 'negative_y', 'country', 'research_site',
       'location_metadata', 'habitat', 'day', 'month', 'year', 'time_hr',
       'time_min', 'age_sex_group', 'site', 'value', 'utm', 'negative',
       'location_count'],
      dtype='object')


In [159]:
train_video_count

,utm,video_count
0,0214665_0056272,14
1,0214762_0051383,6
2,0215546_0053266,5
3,0215578_0053293,7
4,0215958_0050905,50
...,...,...
384,0809321_9882304,4
385,0809842_9882780,18
386,0809865_9882811,40
387,0810284_9882952,7


In [160]:
train_count = train_beh_count.merge(train_video_count, on="utm")
val_count = val_beh_count.merge(val_video_count, on="utm")

In [161]:
def count_unique_behaviours_per_row(row):
    # iterate over each row and count the number of unique behaviours
    beh_counter = 0
    for i in range(0, 13):
        if int(row.iloc[i]) > 0:
            beh_counter += 1
        
    return beh_counter

train_count["behaviour_count"] = train_count.apply(count_unique_behaviours_per_row, axis=1)
# remove columns 0, 1 ... 13
train_count = train_count.drop(columns=[str(i) for i in range(0, 14)])
val_count["behaviour_count"] = val_count.apply(count_unique_behaviours_per_row, axis=1)
val_count = val_count.drop(columns=[str(i) for i in range(0, 14)])

In [162]:
train_count

,utm,video_count,behaviour_count
0,0381601_0967121,4,6
1,0337403_0191785,11,4
2,0381606_0967128,59,9
3,0340963_1432403,18,4
4,0337088_0191451,28,6
...,...,...,...
384,0744557_9558875,1,1
385,0343048_1434623,1,1
386,0744657_9564054,1,1
387,0522272_0587476,1,1


In [163]:
val_count

,utm,video_count,behaviour_count
0,0381601_0967121,3,4
1,0337403_0191785,5,5
2,0381606_0967128,39,10
3,0340963_1432403,13,7
4,0231560_9396216,8,4
...,...,...,...
259,0686477_0649112,1,3
260,0256165_1293907,1,2
261,0486023_0569733,2,4
262,0687357_0648673,2,5


In [164]:
# sort by unique behaviours and then by total count
train_count = train_count.sort_values(
    by=[ "video_count", "behaviour_count",], ascending=[False, False]
)

val_count = val_count.sort_values(
    by=["video_count", "behaviour_count", ], ascending=[False, False]
)

In [165]:
val_count

,utm,video_count,behaviour_count
96,0216558_0056315,99,10
2,0381606_0967128,39,10
15,0336939_0191998,31,8
21,0336939_0191979,28,10
13,0381564_0967028,22,9
...,...,...,...
234,0782167_0812105,1,1
242,0660653_0606290,1,1
248,0696798_0596071,1,1
254,0744661_9561056,1,1


In [166]:
# select video count and utm for separate df
train_video_count = train_count[["utm", "video_count"]].sort_values(by="video_count", ascending=False)
val_video_count = val_count[["utm", "video_count"]].sort_values(by="video_count", ascending=False)


train_beh_count = train_count[["utm", "behaviour_count"]].sort_values(by="behaviour_count", ascending=False)
val_beh_count = val_count[["utm", "behaviour_count"]].sort_values(by="behaviour_count", ascending=False)

In [167]:
def return_beh_segments(df_count, head=50, tail=10, return_cumsum_df=True, total_behaviours = 14):
    df_count["beh_coverage"] = (df_count["behaviour_count"] / total_behaviours) * 100

    # Select locations that make up 50% of the data
    head_df = df_count.query(f"beh_coverage >= {head}")

    # Calculate locations outside the top 50% with more than 10 samples
    tail_df = df_count.query(f"beh_coverage < {head} & behaviour_count >= {tail}")

    # Calculate locations with fewer than 10 samples
    few_shot_df = df_count.query(f"behaviour_count < {tail}")
    if return_cumsum_df:
        return head_df, tail_df, few_shot_df
    return (
        head_df["utm"].values.tolist(),
        tail_df["utm"].values.tolist(),
        few_shot_df["utm"].values.tolist(),
    )



def return_cam_segments(df_count, head=50, tail=10, return_cumsum_df=True):
    # rename columns
    total_videos = df_count["video_count"].sum()

    df_count["cumulative_count"] = df_count["video_count"].cumsum()
    df_count["cumulative_percentage"] = (
        df_count["cumulative_count"] / total_videos
    ) * 100

    # Select locations that make up 50% of the data
    head_df = df_count[df_count["cumulative_percentage"] <= head]

    # Calculate locations outside the top 50% with more than 10 samples
    tail_df = df_count[df_count["cumulative_percentage"] > head]
    tail_df = tail_df[tail_df["video_count"] >= tail]

    # Calculate locations with fewer than 10 samples
    few_shot_df = df_count[df_count["video_count"] < tail]
    if return_cumsum_df:
        return head_df, tail_df, few_shot_df
    return (
        head_df["utm"].values.tolist(),
        tail_df["utm"].values.tolist(),
        few_shot_df["utm"].values.tolist(),
    )


In [168]:
#head_df, tail_df, few_shot_df = return_segments_new(
#    train_count, head=50, tail=10, head_beh=40, beh_tail=3
#)

head_beh_df, tail_beh_df, few_shot_beh_df = return_beh_segments(
    train_beh_count, head=50, tail=3, total_behaviours=11
)

head_cam_df, tail_cam_df, few_shot_cam_df = return_cam_segments(train_video_count, head=50, tail=5)


In [169]:
head_beh_df.shape[0], tail_beh_df.shape[0], few_shot_beh_df.shape[0]


(64, 128, 197)

In [170]:
head_cam_df.shape[0], tail_cam_df.shape[0], few_shot_cam_df.shape[0]

(51, 125, 213)

In [171]:
head_beh_df

,utm,behaviour_count,beh_coverage
38,0544595_0819433,11,100.000000
13,0381564_0967028,10,90.909091
233,0544525_0819848,10,90.909091
170,0607416_0248474,10,90.909091
89,0216558_0056315,10,90.909091
...,...,...,...
173,0605619_0244636,6,54.545455
71,0745656_9557749,6,54.545455
68,0216572_0056318,6,54.545455
113,0605372_0249583,6,54.545455


In [172]:
tail_cam_df

,utm,video_count,cumulative_count,cumulative_percentage
254,0806023_9883895,13,1434,50.351124
47,0685375_0649767,13,1447,50.807584
34,0605137_0249028,12,1459,51.228933
180,0548309_0822019,12,1471,51.650281
174,0544176_0818106,12,1483,52.071629
...,...,...,...,...
177,0544696_0820080,5,2339,82.127809
187,0508234_0689577,5,2344,82.303371
190,0554325_9770544,5,2349,82.478933
215,0231366_9394495,5,2354,82.654494


In [173]:
head_train_beh_list = head_beh_df["utm"].values.tolist()
tail_train_beh_list = tail_beh_df["utm"].values.tolist()
few_shot_train_beh_list = few_shot_beh_df["utm"].values.tolist()

head_train_cam_list = head_cam_df["utm"].values.tolist()
tail_train_cam_list = tail_cam_df["utm"].values.tolist()
few_shot_train_cam_list = few_shot_cam_df["utm"].values.tolist()

In [174]:
def calculate_metrics(df, round_to=3, show_per_class=False):
    f1_values = multilabel_f1_score(
        torch.tensor(np.stack(df["pred"])),
        torch.tensor(np.stack(df["label"])),
        num_labels=14,
        average="none",
    )
    # avg_map = round(map_values.mean().item(), round_to)
    avg_f1 = round(f1_values.mean().item(), round_to)
    if show_per_class:
        val_list = []

        for v in f1_values:
            val_list.append(round(v.item(), round_to))

        # return map_values
        return val_list
    return avg_f1

In [175]:
th_beh_df = train_df[train_df["utm"].isin(head_train_beh_list)]
tt_beh_df = train_df[train_df["utm"].isin(tail_train_beh_list)]
tf_beh_df = train_df[train_df["utm"].isin(few_shot_train_beh_list)]

vh_beh_df = val_df[val_df["utm"].isin(head_train_beh_list)]
vt_beh_df = val_df[val_df["utm"].isin(tail_train_beh_list)]
vf_beh_df = val_df[val_df["utm"].isin(few_shot_train_beh_list)]


th_cam_df = train_df[train_df["utm"].isin(head_train_cam_list)]
tt_cam_df = train_df[train_df["utm"].isin(tail_train_cam_list)]
tf_cam_df = train_df[train_df["utm"].isin(few_shot_train_cam_list)]

vh_cam_df = val_df[val_df["utm"].isin(head_train_cam_list)]
vt_cam_df = val_df[val_df["utm"].isin(tail_train_cam_list)]
vf_cam_df = val_df[val_df["utm"].isin(few_shot_train_cam_list)]

In [176]:
avg_map_beh_th = calculate_metrics(th_beh_df)
avg_map_beh_tt = calculate_metrics(tt_beh_df)
avg_map_beh_tf = calculate_metrics(tf_beh_df)


avg_map_beh_vh = calculate_metrics(vh_beh_df)
avg_map_beh_vt = calculate_metrics(vt_beh_df)
avg_map_beh_vf = calculate_metrics(vf_beh_df)


map_values_beh_th = calculate_metrics(th_beh_df, show_per_class=True)
map_values_beh_tt = calculate_metrics(tt_beh_df, show_per_class=True)
map_values_beh_tf = calculate_metrics(tf_beh_df, show_per_class=True)

map_values_beh_vh = calculate_metrics(vh_beh_df, show_per_class=True)
map_values_beh_vt = calculate_metrics(vt_beh_df, show_per_class=True)
map_values_beh_vf = calculate_metrics(vf_beh_df, show_per_class=True)

# show as dataframe
result_beh_avg = pd.DataFrame(
    {
        "head": [avg_map_beh_th, avg_map_beh_vh],
        "tail": [avg_map_beh_tt, avg_map_beh_vt],
        "few_shot": [avg_map_beh_tf, avg_map_beh_vf],
    },
    index=["train", "val"],
)

result_beh_avg

,head,tail,few_shot
train,0.946,0.947,0.457
val,0.508,0.424,0.226


In [177]:
avg_map_cam_th = calculate_metrics(th_cam_df)
avg_map_cam_tt = calculate_metrics(tt_cam_df)
avg_map_cam_tf = calculate_metrics(tf_cam_df)

avg_map_cam_vh = calculate_metrics(vh_cam_df)
avg_map_cam_vt = calculate_metrics(vt_cam_df)
avg_map_cam_vf = calculate_metrics(vf_cam_df)

map_values_cam_th = calculate_metrics(th_cam_df, show_per_class=True)
map_values_cam_tt = calculate_metrics(tt_cam_df, show_per_class=True)
map_values_cam_tf = calculate_metrics(tf_cam_df, show_per_class=True)

map_values_cam_vh = calculate_metrics(vh_cam_df, show_per_class=True)
map_values_cam_vt = calculate_metrics(vt_cam_df, show_per_class=True)
map_values_cam_vf = calculate_metrics(vf_cam_df, show_per_class=True)

# show as dataframe
result_cam_avg = pd.DataFrame(
    {
        "head": [avg_map_cam_th, avg_map_cam_vh],
        "tail": [avg_map_cam_tt, avg_map_cam_vt],
        "few_shot": [avg_map_cam_tf, avg_map_cam_vf],
    },
    index=["train", "val"],
)

result_cam_avg

,head,tail,few_shot
train,0.938,0.965,0.946
val,0.505,0.490,0.331


In [178]:
h_head_train_beh_cam_list = list(set(head_train_cam_list).intersection(head_train_beh_list))
h_tail_train_beh_cam_list = list(set(head_train_cam_list).intersection(tail_train_beh_list))
h_few_shot_train_beh_cam_list = list(set(head_train_cam_list).intersection(few_shot_train_beh_list))

t_head_train_beh_cam_list = list(set(tail_train_cam_list).intersection(head_train_beh_list))
t_tail_train_beh_cam_list = list(set(tail_train_cam_list).intersection(tail_train_beh_list))
t_few_shot_train_beh_cam_list = list(set(tail_train_cam_list).intersection(few_shot_train_beh_list))

f_head_train_beh_cam_list = list(set(few_shot_train_cam_list).intersection(head_train_beh_list))
f_tail_train_beh_cam_list = list(set(few_shot_train_cam_list).intersection(tail_train_beh_list))
f_few_shot_train_beh_cam_list = list(set(few_shot_train_cam_list).intersection(few_shot_train_beh_list))

th_head_beh_cam_df = train_df[train_df["utm"].isin(h_head_train_beh_cam_list)]
th_tail_beh_cam_df = train_df[train_df["utm"].isin(h_tail_train_beh_cam_list)]
th_few_shot_beh_cam_df = train_df[train_df["utm"].isin(h_few_shot_train_beh_cam_list)]

tt_head_beh_cam_df = train_df[train_df["utm"].isin(t_head_train_beh_cam_list)]
tt_tail_beh_cam_df = train_df[train_df["utm"].isin(t_tail_train_beh_cam_list)]
tt_few_shot_beh_cam_df = train_df[train_df["utm"].isin(t_few_shot_train_beh_cam_list)]

tf_head_beh_cam_df = train_df[train_df["utm"].isin(f_head_train_beh_cam_list)]
tf_tail_beh_cam_df = train_df[train_df["utm"].isin(f_tail_train_beh_cam_list)]
tf_few_shot_beh_cam_df = train_df[train_df["utm"].isin(f_few_shot_train_beh_cam_list)]

vh_head_beh_cam_df = val_df[val_df["utm"].isin(h_head_train_beh_cam_list)]
vh_tail_beh_cam_df = val_df[val_df["utm"].isin(h_tail_train_beh_cam_list)]
vh_few_shot_beh_cam_df = val_df[val_df["utm"].isin(h_few_shot_train_beh_cam_list)]

vt_head_beh_cam_df = val_df[val_df["utm"].isin(t_head_train_beh_cam_list)]
vt_tail_beh_cam_df = val_df[val_df["utm"].isin(t_tail_train_beh_cam_list)]
vt_few_shot_beh_cam_df = val_df[val_df["utm"].isin(t_few_shot_train_beh_cam_list)]

vf_head_beh_cam_df = val_df[val_df["utm"].isin(f_head_train_beh_cam_list)]
vf_tail_beh_cam_df = val_df[val_df["utm"].isin(f_tail_train_beh_cam_list)]
vf_few_shot_beh_cam_df = val_df[val_df["utm"].isin(f_few_shot_train_beh_cam_list)]


#print(th_few_shot_beh_cam_df)
#print(tt_few_shot_beh_cam_df)
#print(vh_few_shot_beh_cam_df)
#print(vt_few_shot_beh_cam_df)


avg_map_beh_cam_th_few_shot = 0
avg_map_beh_cam_tf_head = 0
avg_map_beh_cam_vh_few_shot = 0
avg_map_beh_cam_vf_head = 0
#avg_map_beh_cam_tt_few_shot = 0
#avg_map_beh_cam_vt_few_shot = 0

# calculate metrics
avg_map_beh_cam_th_head = calculate_metrics(th_head_beh_cam_df)
avg_map_beh_cam_th_tail = calculate_metrics(th_tail_beh_cam_df)

if th_few_shot_beh_cam_df.shape[0] > 0:
    avg_map_beh_cam_th_few_shot = calculate_metrics(th_few_shot_beh_cam_df)

avg_map_beh_cam_tt_head = calculate_metrics(tt_head_beh_cam_df)
avg_map_beh_cam_tt_tail = calculate_metrics(tt_tail_beh_cam_df)
avg_map_beh_cam_tt_few_shot = calculate_metrics(tt_few_shot_beh_cam_df)

if tt_few_shot_beh_cam_df.shape[0] > 0:
    avg_map_beh_cam_tt_few_shot = calculate_metrics(tt_few_shot_beh_cam_df)

avg_map_beh_cam_tf_tail = calculate_metrics(tf_tail_beh_cam_df)
avg_map_beh_cam_tf_few_shot = calculate_metrics(tf_few_shot_beh_cam_df)

avg_map_beh_cam_vh_head = calculate_metrics(vh_head_beh_cam_df)
avg_map_beh_cam_vh_tail = calculate_metrics(vh_tail_beh_cam_df)

if vh_few_shot_beh_cam_df.shape[0] > 0:
    avg_map_beh_cam_vh_few_shot = calculate_metrics(vh_few_shot_beh_cam_df)

avg_map_beh_cam_vt_head = calculate_metrics(vt_head_beh_cam_df)
avg_map_beh_cam_vt_tail = calculate_metrics(vt_tail_beh_cam_df)
avg_map_beh_cam_vt_few_shot = calculate_metrics(vt_few_shot_beh_cam_df)

if vt_few_shot_beh_cam_df.shape[0] > 0:
    avg_map_beh_cam_vt_few_shot = calculate_metrics(vt_few_shot_beh_cam_df)

avg_map_beh_cam_vf_tail = calculate_metrics(vf_tail_beh_cam_df)
avg_map_beh_cam_vf_few_shot = calculate_metrics(vf_few_shot_beh_cam_df)



In [179]:
# show as dataframe
result_train_beh_cam_avg = pd.DataFrame(
    {
        "head (7, 14)": [avg_map_beh_cam_th_head, avg_map_beh_cam_th_tail, avg_map_beh_cam_th_few_shot],
        "tail (3, 6)": [avg_map_beh_cam_tt_head, avg_map_beh_cam_tt_tail, avg_map_beh_cam_tt_few_shot],
        "few_shot (1,2)": [avg_map_beh_cam_tf_head, avg_map_beh_cam_tf_tail, avg_map_beh_cam_tf_few_shot],
    },
    index=["head (13, 156)", "tail (6, 13)", "few_shot (1, 5)"],
)

result_val_beh_cam_avg = pd.DataFrame(
    {
        "head (7, 14)": [avg_map_beh_cam_vh_head, avg_map_beh_cam_vh_tail, avg_map_beh_cam_vh_few_shot],
        "tail (3, 6)": [avg_map_beh_cam_vt_head, avg_map_beh_cam_vt_tail, avg_map_beh_cam_vt_few_shot],
        "few_shot (1,2)": [avg_map_beh_cam_vf_head, avg_map_beh_cam_vf_tail, avg_map_beh_cam_vf_few_shot],
    },
    index=["head (13, 156)", "tail (6, 13)", "few_shot (1, 5)"],
)

In [180]:
result_train_beh_cam_avg

,"head (7, 14)","tail (3, 6)","few_shot (1,2)"
"head (13, 156)",0.943,0.967,0.000
"tail (6, 13)",0.638,0.969,0.960
"few_shot (1, 5)",0.000,0.341,0.451


In [181]:
result_val_beh_cam_avg

,"head (7, 14)","tail (3, 6)","few_shot (1,2)"
"head (13, 156)",0.511,0.484,0.000
"tail (6, 13)",0.340,0.451,0.409
"few_shot (1, 5)",0.000,0.185,0.229
